## Homework: Vector Search

In this homework, we will learn more about vector search
and embedding. Like in the module, we will use Qdrant and
fastembed

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

Note: if you want to learn how vector search works under
the hood, check 
[homework 3 from 2024 cohort (questions 1-4)](../../2024/03-vector-search/homework.md)

## Embeddings

Qdrant uses [fastembed](https://github.com/qdrant/fastembed)
under the hood to turn text into vectors. 

We will now explore this library

Make sure it's installed:

```bash
pip install fastembed
```

Import it: 

```python
from fastembed import TextEmbedding
```

## Q1. Embedding the query

Embed the query: `'I just discovered the course. Can I join now?'`.
Use the `'jinaai/jina-embeddings-v2-small-en'` model. 

You should get a numpy array of size 512.

What's the minimal value in this array?

* -0.51
* -0.11
* 0
* 0.51

**Answer: `-0.11`**

In [1]:
import numpy as np
from fastembed import TextEmbedding

In [2]:
model_handle = TextEmbedding("jinaai/jina-embeddings-v2-small-en")
embeddings_generator = model_handle

In [3]:
model_handle.model_name

'jinaai/jina-embeddings-v2-small-en'

In [4]:
documents = ["I just discovered the course. Can I join now?"]

In [5]:
embeddings = list(model_handle.embed(documents))

In [6]:
embeddings[0].shape

(512,)

In [7]:
embeddings[0]

array([-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,  3.92670571e-02,
       -1.41797115e-02, -4.68342854e-02,  2.69862822e-02,  3.49039060e-02,
        1.53419217e-03, -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
       -4.87239498e-02, -9.90923923e-02,  6.02579764e-02, -1.86105379e-02,
       -8.81040139e-03, -3.01994831e-02, -1.42541815e-02, -3.87884212e-02,
       -2.76125662e-02, -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
        8.37959894e-02, -8.87329606e-02, -7.30260625e-02,  5.92846802e-02,
        4.07805674e-02,  7.72684822e-02, -4.60029697e-02,  3.98426895e-02,
        1.74821510e-02,  8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
        4.76479896e-02,  1.76566196e-02, -3.62013627e-02, -3.53408062e-02,
       -5.92537995e-03,  2.97091202e-02,  8.05883752e-02,  1.07132722e-02,
       -5.38192519e-02, -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
        7.05763168e-03,  7.70159791e-03, -5.17482529e-02,  5.46955574e-02,
       -7.80289904e-02,  

In [8]:
embeddings[0].min()

np.float64(-0.11726373885183883)

## Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the `norm` function:

```python
import numpy as np
np.linalg.norm(q)
```

Which means that we can simply compute the dot product between
two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

```python
q.dot(q)
```

## Q2. Cosine similarity with another vector

Now let's embed this document:

```python
doc = 'Can I still join the course after the start date?'
```

What's the cosine similarity between the vector for the query
and the vector for the document?

* 0.3
* 0.5
* 0.7
* 0.9

**Answer: `0.9`**

In [9]:
np.linalg.norm(embeddings[0])

np.float64(1.0)

In [10]:
embeddings[0].dot(embeddings[0])

np.float64(1.0000000000000002)

In [11]:
doc = 'Can I still join the course after the start date?'

In [12]:
query_vector = list(model_handle.embed(doc))

In [13]:
query_vector[0].shape

(512,)

In [14]:
# cosine similarity
query_vector[0].dot(embeddings[0])

np.float64(0.9008528895674548)

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

```python
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]
```

Compute the embeddings for the text field, and compute the 
cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

- 0
- 1
- 2
- 3
- 4

Hint: if you put all the embeddings of the text field in one matrix `V` (a single 2-dimensional numpy array), then
computing the cosine becomes a matrix multiplication:

```python
V.dot(q)
```

If this hint is rather confusing you than helping, feel free
to ignore it.

**Answer: `1`**

In [15]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [16]:
query = "I just discovered the course. Can I join now?"

In [17]:
text_documents = [doc['text'] for doc in documents]

In [18]:
text_documents

["Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'You can start by instal

In [19]:
document_embeddings = list(model_handle.embed(text_documents))

In [20]:
query_embedding = list(model_handle.embed([query]))[0]

In [21]:
V = np.stack(document_embeddings)
V.shape

(5, 512)

In [22]:
# Normalize the vectors for cosine similarity
# Cosine similarity = dot product of normalized vectors
V_normalized = V / np.linalg.norm(V, axis=1, keepdims=True)
q_normalized = query_embedding / np.linalg.norm(query_embedding)

In [23]:
cosine_similarities = V_normalized.dot(q_normalized)

for i, similarity in enumerate(cosine_similarities):
    print(f"Document {i}: {similarity:.4f}")

Document 0: 0.7630
Document 1: 0.8182
Document 2: 0.8085
Document 3: 0.7133
Document 4: 0.7304


In [24]:
# Find the index with highest similarity
max_index = np.argmax(cosine_similarities)
max_similarity = cosine_similarities[max_index]
print(f"index: {max_index}, max_similarity: {max_similarity}")

index: 1, max_similarity: 0.8182378150042889


## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of
`question` and `text`:

```python
full_text = doc['question'] + ' ' + doc['text']
``` 

Embed this field and compute the cosine between it and the
query vector. What's the highest scoring document?

- 0
- 1
- 2
- 3
- 4

Is it different from Q3? If yes, why?

**Answer: `0`**

- Reason:
    - The question field often contains the main topic/keywords
    - Concatenating question + text gives more semantic context
    - Better keyword matching between query and question fields
    - The embedding model can capture the semantic relationship more effectively

In [25]:
full_text_documents = []
for i, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    full_text_documents.append(full_text)

In [26]:
len(full_text_documents)

5

In [27]:
full_text_embeddings = list(model_handle.embed(full_text_documents))

In [28]:
query_embedding = list(model_handle.embed([query]))[0]

In [29]:
# Stack full_text embeddings into a matrix V
V = np.stack(full_text_embeddings)
V.shape

(5, 512)

In [30]:
# Normalize the vectors for cosine similarity
V_normalized = V / np.linalg.norm(V, axis=1, keepdims=True)
q_normalized = query_embedding / np.linalg.norm(query_embedding)

In [31]:
# Using matrix multiplication (the hint)
cosine_similarities = V_normalized.dot(q_normalized)

In [32]:
for i, similarity in enumerate(cosine_similarities):
    print(f"Document {i}: {similarity:.4f}")

Document 0: 0.8515
Document 1: 0.8437
Document 2: 0.8408
Document 3: 0.7755
Document 4: 0.8086


In [33]:
# Find the index with highest similarity
max_index = np.argmax(cosine_similarities)
max_similarity = cosine_similarities[max_index]
print(f"index: {max_index}, max_similarity: {max_similarity}")

index: 0, max_similarity: 0.8514543236908068


## Q5. Selecting the embedding model

Now let's select a smaller embedding model.
What's the smallest dimensionality for models in fastembed?

- 128
- 256
- 384
- 512

One of these models is `BAAI/bge-small-en`. Let's use it.

**Answer:`384`**

In [34]:
import json

In [35]:
# First, let's see all unique dimensions available
dimensions = set()
for model in TextEmbedding.list_supported_models():
    dimensions.add(model["dim"])

In [36]:
sorted(dimensions)

[384, 512, 768, 1024]

In [37]:
min(dimensions)

384

In [38]:
SMALLEST_DIMENSIONALITY = min(dimensions)

In [39]:
for model in TextEmbedding.list_supported_models():
    if model["dim"] == SMALLEST_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [40]:
for model in TextEmbedding.list_supported_models():
    if "bge-small-en" in model["model"]:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

```python
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

```python
text = doc['question'] + ' ' + doc['text']
```

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results?
(The score for the first returned record):

- 0.97
- 0.87
- 0.77
- 0.67

**Answer: `0.87`**

In [41]:
import json
import requests
import numpy as np
from fastembed import TextEmbedding
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [42]:
# Download documents
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

In [43]:
# Filter for ML Zoomcamp only
documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [44]:
len(documents)

375

In [45]:
# Use the model from Q5 - let's first find the smallest dimension model
available_dims = set()
for model in TextEmbedding.list_supported_models():
    available_dims.add(model["dim"])

smallest_dim = min(available_dims)
smallest_dim

384

In [46]:
# Find a model with smallest dimension
smallest_models = []
for model in TextEmbedding.list_supported_models():
    if model["dim"] == smallest_dim:
        smallest_models.append(model["model"])

smallest_models

['BAAI/bge-small-en',
 'BAAI/bge-small-en-v1.5',
 'snowflake/snowflake-arctic-embed-xs',
 'snowflake/snowflake-arctic-embed-s',
 'sentence-transformers/all-MiniLM-L6-v2',
 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2']

In [47]:
embedding_model = TextEmbedding("BAAI/bge-small-en")
model_name = "BAAI/bge-small-en"

In [48]:
# Create combined text (question + answer)
texts_to_embed = []
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    texts_to_embed.append(text)

In [49]:
texts_to_embed[0]

'How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork'

In [50]:
# generate embeddings
embeddings = list(embedding_model.embed(texts_to_embed))

In [51]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [52]:
collection_name = "ml_zoomcamp_faq"

In [54]:
client.delete_collection(collection_name=collection_name)

True

In [55]:
# Create collection
client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=smallest_dim, distance=Distance.COSINE),
)

True

In [56]:
# Prepare points for insertion
points = []
for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
    point = PointStruct(
        id=i,
        vector=embedding.tolist(),
        payload={
            "question": doc["question"],
            "text": doc["text"],
            "section": doc["section"],
            "course": doc["course"],
            "combined_text": doc['question'] + ' ' + doc['text']
        }
    )
    points.append(point)

In [57]:
# Insert points in batches
batch_size = 100
for i in range(0, len(points), batch_size):
    batch = points[i:i+batch_size]
    client.upsert(
        collection_name=collection_name,
        points=batch
    )

In [58]:
# Query from Q1
query = "I just discovered the course. Can I join now?"

In [59]:
# Generate query embedding
query_embedding = list(embedding_model.embed([query]))[0]

In [60]:
# Search in Qdrant
search_results = client.query_points(
    collection_name=collection_name,
    query=query_embedding.tolist(),
    limit=5  # Get top 5 results
)

In [61]:
search_results

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'question': 'The course has already started. Can I still join it?', 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp', 'combined_text': 'The course has already started. Can I still join it? Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work o

In [63]:
print("Top 5 search results:")
for i, result in enumerate(search_results.points[:5]):
    print(f"\n{i+1}. Score: {result.score:.4f}")
    print(f"   Question: {result.payload['question']}")
    print(f"   Text: {result.payload['text'][:150]}...")
    print(f"   Section: {result.payload['section']}")

Top 5 search results:

1. Score: 0.8703
   Question: The course has already started. Can I still join it?
   Text: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to...
   Section: General course-related questions

2. Score: 0.8692
   Question: How long is the course?
   Text: Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)...
   Section: General course-related questions

3. Score: 0.8683
   Question: I’m new to Slack and can’t find the course channel. Where is it?
   Text: Here’s how you join a in Slack: https://slack.com/help/articles/205239967-Join-a-channel
Click “All channels” at the top of your left sidebar. If you ...
   Section: General course-related questions

4. Score: 0.8576
   Question: How to get started with Week 10?
   Text: TODO...
   Section: 10. Kubernetes and TensorFlow Serving

5. Score: 0.8572
   

In [64]:
# Get the highest score from the list of ScoredPoint objects
highest_score = max(point.score for point in search_results.points)
print(f"Highest score: {highest_score:.4f}")

Highest score: 0.8703
